## 항공가격 크롤링을 위한 1차 크롤링 작업
- 대상 사이트
    + 실시간 : 인터파크, 웹투어
    + 항공사 : 대한항공, 아시아나, 제주항공, 진에어, 에어부산, 티웨이
    + 실행 시간 : 매일 아침 출근후 실행
    + 실행 노선 : 김포 - 제주 내일부터 60일간, 인천 - 나리타 내일부터 60일간
    + 데이터 : 사이트를 스크래핑한 raw text를 텍스트 파일로 저장
        * 저장시 헤더 정보 생성 및 저장 : 저장 포맷 [스크래핑 날짜 시간, 사이트, 조회조건]
    + 파일 형식
        * 실시간 : scrap_YYMMDDHH_대상사이트코드_항공사코드_노선[출발-도착]_출발일.txt
            * 예)scrap_2017052415_IP_LJ_GMP_CJU_20170610.txt
        * 항공사 : scrap_YYMMDDHH_대상사이트코드_노선[출발-도착]_출발일.txt
            * 예)scrap_2017052415_KE_GMP_CJU_20170610.txt
        * 사이트 코드
            * IP - 인터파크, WP - 웹투어, 각 항공사 2자리 코드(KE,OZ,7C,LJ,BX,TW)
    + 로그 처리 형식 - 고민 필요
        * 포맷
        * 오류 로그 :
    + 크롤링 결과물 체크 방식
        * 여타 이유로 결과가 없을 경우 처리
            - 1차 크롤링 후 분석 시 데이터가 없을 경우 만 취합 하여 추가 크롤링을 진행 할 수 있도록 처리 필요
            - 데이터 처리를 위한 폴더 구조
                + crawl : 1차 크롤링 raw data 저장 폴더
                + parsed_data : 파싱된 형태의 데이터를 저장, csv 형식 파일 형식으로 분석된 데이터 저장
                + parsed_ok : 정상적으로 파싱된 raw data 파일 저장
                + parsed_error : 문제가 있는 raw data 파일 저장

### 기준정보 정의

- 사이트 코드 정보, 대상 노선 정보, 노선과 사이트 코드 정보 매핑 테이블
- 크롤링 기간 정보
- 실시간 여행사 정보 스크래핑시 대상 항공사 코드

### 로그 처리
- log_util 의 common_logger 이용하여 로그 처리
-  log(messages,logger,level)

In [1]:
from collections import namedtuple
from datetime import datetime, timedelta
import requests
from common.env_variable import *
from common.batch_util import *
from common.crawl_func import *
from common.log_util import *
from common.util import save_raw_data
## 로그 초기화
logger_initialize('crawl_logger_setting.json')

### 국내선/국제선 크롤링 배치 함수
1. 세팅 정보중 국내선 사이트 정보 읽어오기
2. 해당 정보를 이용 각 사이트 크롤링 진행
    1. 기간 생성
    2. 해당 기간 동안 노선 정보 만큼 크롤링 진행
        * 진행시 항공사 여부 체크하여 항공사가 아니면 스크래핑 대상 항공사 코드 정보 체크
    3. 헤더 생성 후 파일로 저장

In [2]:
## 세팅 파일을 이용한 크롤링 배치 함수
## 국내선/국제선 구분 0-국내선,1-국제선
def batch_crawl(dom_int='0',interval=None):
    start_time = datetime.today()
    log_msgs = ['start batch job','crawling domestic site']
    log(log_msgs,level=logging.INFO)
    crawl_cnt = 0
    for site_code in CRAWL_SITE_CODES:
        log(site_code,level=logging.INFO)
        if site_code is None: ## None 아닌 경우만 진행
            continue
        ## 국내선/국제선 체크, None 이면 모두
        if dom_int is not None and site_code.dom_int != dom_int:
            continue
        func = None
        try:
            func = eval(site_code.func)
        except NameError as e:
            log(e,level=logging.ERROR)
        if func is None: ## 해당 함수가 없을 경우 다음으로
            continue
        ## 해당 함수 이용 크롤링 진행
        if interval is None:
            interval = CRAWL_INTERVAL
        for dpt_date in crawl_period(interval):
            ## 크롤링 노선 만큼 크롤링 진행
            for route_code,route_info in CRAWL_ROUTES.items():
                log([route_code,ROUTE_SITE_MAP[route_code]],level=logging.DEBUG)
                if route_info.dom_int != site_code.dom_int: ## 국내선/국제선 체크
                    continue
                ## 맵체크 값이 없으면 패스
                if site_code.site not in ROUTE_SITE_MAP[route_code]:                   
                    continue
                if site_code.isairline: ## 항공사 사이트
                    raw_data = crawling_func(func,route_info.dpt_3p,route_info.arr_3p,dpt_date)
                    head = set_headinfo(site_code.site,site_code.dom_int,site_code.site,route_info.dpt_3p,route_info.arr_3p,dpt_date)
                    file = file_name(site_code.site,route_info.dpt_3p,route_info.arr_3p,dpt_date)
                    save_raw_data(file,raw_data,head=head)
                    crawl_cnt += 1
                else: ## 실시간 사이트
                    for airline in TARGET_AIRLINES:
                        ## 조회대상 항공사가 맵에 포함되지 않을 경우 체크
                        if airline not in ROUTE_SITE_MAP[route_code]:
                            continue
                        raw_data = crawling_func(func,airline,route_info.dpt_3p,route_info.arr_3p,dpt_date)
                        head = set_headinfo(site_code.site,site_code.dom_int,airline,route_info.dpt_3p,route_info.arr_3p,dpt_date)
                        file = file_name(site_code.site,route_info.dpt_3p,route_info.arr_3p,dpt_date,airline=airline)
                        save_raw_data(file,raw_data,head=head)
                        crawl_cnt += 1
    end_time = datetime.today()
    log_msgs = ['end batch job','crawling domestic site',
                'elapsed -{}'.format(end_time-start_time),'Total crawl:{} files saved.'.format(crawl_cnt)]
    log(log_msgs,level=logging.INFO)

In [11]:
## 배치 함수 실행
## '0' : 국내선, '1' : 국제선, None : 모두
batch_crawl(dom_int='0',interval=60)

2017-06-29 08:25:28,446 root     INFO     start batch job :: crawling domestic site
2017-06-29 08:25:28,449 root     INFO     0 :: IP :: crawling_IP_dom :: False
2017-06-29 08:25:28,453 root     DEBUG    ICN-NRT :: ['IP', '7C', 'TW', 'LJ', 'ZE']
2017-06-29 08:25:28,458 root     DEBUG    GMP-CJU :: ['IP', '7C', 'TW', 'LJ', 'BX', 'ZE']
2017-06-29 08:25:28,462 root     DEBUG    Crawling Interpark domastic schedule site
2017-06-29 08:25:28,519 root     DEBUG    Crawling Interpark domastic schedule site


Start Simple crawling :  http://domair.interpark.com/api/booking/airJourney.do
>> Parameters
chd:0 , dep:GMP , adt:1 , tripDivi:0 , inf:0 , airlineCode:7C , depDate:20170630 , arr:CJU , format:json , 
End Simple crawling
Start Simple crawling :  http://domair.interpark.com/api/booking/airJourney.do
>> Parameters
chd:0 , dep:GMP , adt:1 , tripDivi:0 , inf:0 , airlineCode:TW , depDate:20170630 , arr:CJU , format:json , 


2017-06-29 08:25:29,114 root     DEBUG    Crawling Interpark domastic schedule site
2017-06-29 08:25:29,290 root     DEBUG    Crawling Interpark domastic schedule site


End Simple crawling
Start Simple crawling :  http://domair.interpark.com/api/booking/airJourney.do
>> Parameters
chd:0 , dep:GMP , adt:1 , tripDivi:0 , inf:0 , airlineCode:LJ , depDate:20170630 , arr:CJU , format:json , 
End Simple crawling
Start Simple crawling :  http://domair.interpark.com/api/booking/airJourney.do
>> Parameters
chd:0 , dep:GMP , adt:1 , tripDivi:0 , inf:0 , airlineCode:BX , depDate:20170630 , arr:CJU , format:json , 


2017-06-29 08:25:29,922 root     DEBUG    Crawling Interpark domastic schedule site


...

2017-06-29 08:26:37,573 root     DEBUG    Crawling Interpark domastic schedule site
2017-06-29 08:26:37,758 root     DEBUG    Crawling Interpark domastic schedule site


End Simple crawling
Start Simple crawling :  http://domair.interpark.com/api/booking/airJourney.do
>> Parameters
chd:0 , dep:GMP , adt:1 , tripDivi:0 , inf:0 , airlineCode:LJ , depDate:20170725 , arr:CJU , format:json , 
End Simple crawling
Start Simple crawling :  http://domair.interpark.com/api/booking/airJourney.do
>> Parameters
chd:0 , dep:GMP , adt:1 , tripDivi:0 , inf:0 , airlineCode:BX , depDate:20170725 , arr:CJU , format:json , 


2017-06-29 08:26:38,383 root     DEBUG    Crawling Interpark domastic schedule site


End Simple crawling
Start Simple crawling :  http://domair.interpark.com/api/booking/airJourney.do
>> Parameters
chd:0 , dep:GMP , adt:1 , tripDivi:0 , inf:0 , airlineCode:ZE , depDate:20170725 , arr:CJU , format:json , 
$$ Check : Unexpected error:


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))

In [3]:
# 특정 사이트 배치 처리 함수
def batch_onesite_crawl(site,dom_int='0',interval=None):
    if site == None:
        return None
    start_time = datetime.today()
    log_msgs = ['start batch job','crawling domestic site']
    log(log_msgs,level=logging.INFO)
    crawl_cnt = 0
    func, isairline = get_crawl_site_func(dom_int,site)
    log([func,isairline],level=logging.INFO)
    if func is None: ## None 아닌 경우만 진행
        return None
    ## 해당 함수 이용 크롤링 진행
    if interval is None:
        interval = CRAWL_INTERVAL
    for dpt_date in crawl_period(interval):
        ## 크롤링 노선 만큼 크롤링 진행
        for route_code,route_info in CRAWL_ROUTES.items():
            log([route_code,ROUTE_SITE_MAP[route_code]],level=logging.DEBUG)
            if route_info.dom_int != dom_int: ## 국내선/국제선 체크
                continue
            ## 맵체크 값이 없으면 패스
            if site not in ROUTE_SITE_MAP[route_code]:                   
                continue
            if isairline: ## 항공사 사이트
                raw_data = crawling_func(func,route_info.dpt_3p,route_info.arr_3p,dpt_date)
                head = set_headinfo(site,dom_int,site,route_info.dpt_3p,route_info.arr_3p,dpt_date)
                file = file_name(site,route_info.dpt_3p,route_info.arr_3p,dpt_date)
                save_raw_data(file,raw_data,head=head)
                crawl_cnt += 1
            else: ## 실시간 사이트
                for airline in TARGET_AIRLINES:
                    ## 조회대상 항공사가 맵에 포함되지 않을 경우 체크
                    if airline not in ROUTE_SITE_MAP[route_code]:
                        continue
                    raw_data = crawling_func(func,airline,route_info.dpt_3p,route_info.arr_3p,dpt_date)
                    head = set_headinfo(site,dom_int,airline,route_info.dpt_3p,route_info.arr_3p,dpt_date)
                    file = file_name(site,route_info.dpt_3p,route_info.arr_3p,dpt_date,airline=airline)
                    save_raw_data(file,raw_data,head=head)
                    crawl_cnt += 1
    end_time = datetime.today()
    log_msgs = ['end batch job','crawling domestic site',
                'elapsed -{}'.format(end_time-start_time),'Total crawl:{} files saved.'.format(crawl_cnt)]
    log(log_msgs,level=logging.INFO)

In [4]:
## 특정 사이트 배치 함수 실행
batch_onesite_crawl("IP",dom_int='1',interval=60)

2017-06-29 09:10:44,720 root     INFO     start batch job :: crawling domestic site
2017-06-29 09:10:44,740 root     DEBUG    1 :: IP :: crawling_IP_int :: False
2017-06-29 09:10:44,742 root     INFO     <function crawling_IP_int at 0x0000000007F6A2F0> :: False
2017-06-29 09:10:44,745 root     DEBUG    ICN-NRT :: ['IP', '7C', 'TW', 'LJ', 'ZE']
2017-06-29 09:10:44,750 root     DEBUG    Crawling Interpark domastic schedule site


Start Simple crawling :  http://smartair.interpark.com/HtmlSearch/GetGoodsSmartList.aspx
>> Parameters
adt:1 , comp:Y , Soto:N , Change: , BEST:Y , depdate0:20170630 , chd:0 , FLEX:N , StayLength: , ptype:I , dep0:ICN , SeatAvail:Y , AirLine:7C , JSON:Y , inf:0 , SeatType:A , enc:u , arr0:NRT , SplitNo:100 , trip:OW , 


2017-06-29 09:10:45,422 root     DEBUG    Crawling Interpark domastic schedule site
2017-06-29 09:10:45,528 root     DEBUG    Crawling Interpark domastic schedule site


End Simple crawling
Start Simple crawling :  http://smartair.interpark.com/HtmlSearch/GetGoodsSmartList.aspx
>> Parameters
adt:1 , comp:Y , Soto:N , Change: , BEST:Y , depdate0:20170630 , chd:0 , FLEX:N , StayLength: , ptype:I , dep0:ICN , SeatAvail:Y , AirLine:TW , JSON:Y , inf:0 , SeatType:A , enc:u , arr0:NRT , SplitNo:100 , trip:OW , 
End Simple crawling
Start Simple crawling :  http://smartair.interpark.com/HtmlSearch/GetGoodsSmartList.aspx
>> Parameters
adt:1 , comp:Y , Soto:N , Change: , BEST:Y , depdate0:20170630 , chd:0 , FLEX:N , StayLength: , ptype:I , dep0:ICN , SeatAvail:Y , AirLine:LJ , JSON:Y , inf:0 , SeatType:A , enc:u , arr0:NRT , SplitNo:100 , trip:OW , 


2017-06-29 09:10:45,676 root     DEBUG    Crawling Interpark domastic schedule site


...

End Simple crawling
Start Simple crawling :  http://smartair.interpark.com/HtmlSearch/GetGoodsSmartList.aspx
>> Parameters
adt:1 , comp:Y , Soto:N , Change: , BEST:Y , depdate0:20170828 , chd:0 , FLEX:N , StayLength: , ptype:I , dep0:ICN , SeatAvail:Y , AirLine:LJ , JSON:Y , inf:0 , SeatType:A , enc:u , arr0:NRT , SplitNo:100 , trip:OW , 


2017-06-29 09:18:00,214 root     DEBUG    Crawling Interpark domastic schedule site


End Simple crawling
Start Simple crawling :  http://smartair.interpark.com/HtmlSearch/GetGoodsSmartList.aspx
>> Parameters
adt:1 , comp:Y , Soto:N , Change: , BEST:Y , depdate0:20170828 , chd:0 , FLEX:N , StayLength: , ptype:I , dep0:ICN , SeatAvail:Y , AirLine:ZE , JSON:Y , inf:0 , SeatType:A , enc:u , arr0:NRT , SplitNo:100 , trip:OW , 


2017-06-29 09:18:02,276 root     DEBUG    GMP-CJU :: ['IP', '7C', 'TW', 'LJ', 'BX', 'ZE']
2017-06-29 09:18:02,278 root     INFO     end batch job :: crawling domestic site :: elapsed -0:07:17.558026 :: Total crawl:240 files saved.


End Simple crawling
